In [1]:
import influxdb_client, os, time
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

import numpy as np
import pandas as pd


# Read csv file into pd dataFrame

In [8]:
csvfile = "../../data-connectors/influxdb_connector/venv/influxdb_test_date_filled.csv"

df = pd.read_csv(csvfile, sep=",", engine='python',index_col=False)

In [9]:
display(df[:5])

,time,Accelerometer1RMS,Accelerometer2RMS,Current,Pressure,Temperature,Thermocouple,Voltage,Volume Flow RateRMS,anomaly,changepoint,status
0,9/1/2022 13:00,0.081056,0.132572,1.40948,NaN,NaN,22.0867,231.247,77.0000,0.0,0.0,inactive
1,9/1/2022 13:00,0.081417,0.136616,1.94200,NaN,NaN,22.0904,229.569,76.9794,NaN,NaN,active
2,9/1/2022 13:00,0.082209,0.132413,1.60694,0.054711,86.7442,NaN,NaN,76.0209,0.0,NaN,active
3,9/1/2022 13:00,NaN,0.137628,1.13916,NaN,86.8673,22.0944,218.121,77.0000,0.0,0.0,active
4,9/1/2022 13:00,0.081960,0.130911,1.57836,NaN,86.9175,22.0887,231.304,77.0000,NaN,0.0,inactive


# Convert date into linux timestamp. 

In [11]:

df['time'] = pd.to_datetime(df['time'])


print(df.columns)
    


Index(['time', 'Accelerometer1RMS', 'Accelerometer2RMS', 'Current', 'Pressure',
       'Temperature', 'Thermocouple', 'Voltage', 'Volume Flow RateRMS',
       'anomaly', 'changepoint', 'status'],
      dtype='object')


# Initialize InfluxDB

In [2]:
token = "84MNLR5ekgVAQAMHHDFw9vKSy--rYXMZZqqa-H3d-WgNQNO_149mj0zDHDsomUpEJNcOVbNbd7VY5eebNs31MA=="
org = "kris1.pan@intel.com"
url = "https://us-east-1-1.aws.cloud2.influxdata.com"

client = influxdb_client.InfluxDBClient(url=url, token=token, org=org)

# Write df into InfluxDB

In [14]:
bucket="anomaly_detection"
tgs = ['Accelerometer1RMS', 'Accelerometer2RMS', 'Current', 'Pressure',
       'Temperature', 'Thermocouple', 'Voltage', 'Volume Flow RateRMS',
       'anomaly', 'changepoint', 'status']
with InfluxDBClient(url=url, token=token, org=org) as client:
    df =df
    client.write_api(write_options=SYNCHRONOUS).write(bucket=bucket, record=df, data_frame_measurement_name="sensors", data_frame_timestamp_column= "time")
    print('done')

done


# Query Pandas df from InfluxDB

In [17]:
query_api = influxdb_client.InfluxDBClient(url=url, token=token, org=org).query_api()

query = """from(bucket: "anomaly_detection")
 |> range(start: -10y, stop: now()) 
 |> filter(fn: (r) => r._measurement == "sensors") 
 |> filter(fn: (r) => r._field == "Accelerometer1RMS" or r._field == "Accelerometer2RMS" or r._field == "Current" or r._field == "Anomaly" )
 |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
 |> keep(columns: ["_time", "Accelerometer1RMS", "Accelerometer2RMS", "Current", "Anomaly"])"""

weather_data = query_api.query_data_frame(query=query, org="basem.barakat@intel.com")

# display(weather_data.columns)
display(weather_data)

,result,table,_time,Accelerometer1RMS,Accelerometer2RMS,Current,Pressure
0,_result,0,2022-09-01 13:00:00+00:00,0.081630,0.135217,1.96322,-0.601143
1,_result,0,2022-09-01 13:01:00+00:00,0.082191,0.130613,1.45997,0.054711
2,_result,0,2022-09-01 13:02:00+00:00,0.081346,0.135386,1.05278,-0.273216
3,_result,0,2022-09-01 13:03:00+00:00,0.081721,0.136347,1.58415,0.382638
4,_result,0,2022-09-01 13:04:00+00:00,0.082114,0.132453,1.40010,0.054711
5,_result,0,2022-09-01 13:05:00+00:00,0.081144,0.138545,1.71987,-0.273216
6,_result,0,2022-09-01 13:06:00+00:00,0.081262,0.131313,1.57426,0.382638
7,_result,0,2022-09-01 13:07:00+00:00,0.081182,0.130699,1.99300,-0.601143
8,_result,0,2022-09-01 13:08:00+00:00,0.081997,0.135727,1.39849,-0.273216
9,_result,0,2022-09-01 13:09:00+00:00,0.081739,0.135904,1.92894,-0.273216
